In [ ]:
import numpy as np
import pygame
import sys
import math
import random

#deklarasi variabel yang diperlukan
JML_BARIS = 6
JML_KOLOM = 7
KOTAK = 75
PEMAIN = 0
KOMPUTER= 1

#deklarasi warna yang digunakan
BLUE = (0,0,225)
BLACK = (0,0,0)
RED = (225,0,0)
YELLOW = (255,225,0)

#fungsi untuk membuat papan permainan
def buat_papan():
    papan = np.zeros((JML_BARIS, JML_KOLOM), dtype=int)
    return papan

#fungsi untuk menentukan posisi setiap bagian yg dijatuhkan dalam papan
def isi_papan(papan, baris,kolom, isi):
    papan[baris][kolom] = isi

#fungsi untuk memeriksa jika kolom sudah penuh
def cek_penuh(papan, kolom):
    return papan[JML_BARIS-1][kolom] == 0

#fungsi untuk memeriksa apakah baris sudah terisi belum
def cek_visited(papan, kolom):
    for r in range(JML_BARIS):
        if papan[r][kolom] == 0:
            return r
        
#mencetak papan dengan index (0,0) berada pada kiri bawah
def cetak_papan(papan):
    print(np.flip(papan,0))
        
#fungsi untuk memeriksa jika menang
def cek_menang(papan, isi):
    #cek horizontal
    for c in range (JML_KOLOM-3):
        for r in range (JML_BARIS):
            if papan[r][c] == isi and papan[r][c+1] == papan[r][c+2] == papan[r][c+3] == isi:
                return True
            
    #cek vertikal
    for c in range (JML_KOLOM):
        for r in range (JML_BARIS-3):
            if papan[r][c] == isi and papan[r+1][c] == papan[r+2][c] == papan[r+3][c] == isi:
                return True        
    
    #cek diagonal positif
    for c in range (JML_KOLOM-3):
        for r in range (JML_BARIS-3):
            if papan[r][c] == isi and papan[r+1][c+1] == papan[r+2][c+2] == papan[r+3][c+3] == isi:
                return True 
            
    #cek diagonal negatif
    for c in range (JML_KOLOM-3):
        for r in range (3,JML_BARIS):
            if papan[r][c] == isi and papan[r-1][c+1] == papan[r-2][c+2] == papan[r-3][c+3] == isi:
                return True 
    

#fungsi untuk menampilkan dengan pygame        
def draw_board(papan):
    for c in range(JML_KOLOM):
        for r in range (JML_BARIS):
            pygame.draw.rect(screen, BLUE,(c*KOTAK,r*KOTAK+KOTAK, KOTAK, KOTAK))
            pygame.draw.circle(screen, BLACK,(int(c*KOTAK+KOTAK/2),int(r*KOTAK+KOTAK+KOTAK/2)), radius )
    for c in range(JML_KOLOM):
        for r in range (JML_BARIS):
            if papan[r][c] == 1:
                pygame.draw.circle(screen, RED,(int(c*KOTAK+KOTAK/2),int(tinggi-(r*KOTAK+KOTAK/2)) ), radius )
            elif papan[r][c] == 2:
                pygame.draw.circle(screen, YELLOW,(int(c*KOTAK+KOTAK/2),int( tinggi - (r*KOTAK+KOTAK/2) )), radius )
    pygame.display.update()
    

papan = buat_papan()
cetak_papan(papan)
game_over = False
turn = 0
pygame.init()
lebar = JML_KOLOM * KOTAK
tinggi = (JML_BARIS+1)* KOTAK 
size = (lebar,tinggi)
radius = int (KOTAK/2 - 5 )
screen = pygame.display.set_mode(size)
draw_board(papan)
pygame.display.update()
tulisan = pygame.font.SysFont("Arial",50)


while not game_over :
    for event in pygame.event.get():

        #untuk keluar dari tampilan
        if event.type == pygame.QUIT:
            sys.exit()

        #yang ditampilkan pada gerakan mouse
        if event.type == pygame.MOUSEMOTION:
            pygame.draw.rect(screen, BLACK,(0,0, lebar, KOTAK))
            posx = event.pos[0]
            if turn == 0:
                pygame.draw.circle(screen, RED, (posx, int(KOTAK/2)), radius )
        pygame.display.update()


        if event.type == pygame.MOUSEBUTTONDOWN:
            pygame.draw.rect(screen, BLACK,(0,0, lebar, KOTAK))

            if turn == PEMAIN:
                posx = event.pos[0]
                kolom = int(math.floor(posx/KOTAK))          
                if cek_penuh(papan, kolom):
                    baris = cek_visited(papan,kolom)
                    isi_papan(papan, baris,kolom, 1)
                    if cek_menang(papan, 1):
                        #text, antialias,color
                        label = tulisan.render("Pemain menang!!!", 1, RED )
                        screen.blit(label,(40,10))
                        game_over = True
                    turn += 1
                    turn = turn%2
                    cetak_papan(papan)
                    draw_board(papan)
    
    if turn == KOMPUTER and not game_over:
        kolom = random.randint(0,JML_KOLOM-1)
        if cek_penuh(papan, kolom):
            baris = cek_visited(papan,kolom)
            isi_papan(papan, baris,kolom, 2)
            if cek_menang(papan, 2):
                label = tulisan.render("Komputer menang!!!", 1, YELLOW )
                screen.blit(label,(40,10))
                game_over = True
                   
            cetak_papan(papan)
            draw_board(papan)
            turn += 1
            turn = turn%2
            
    if game_over:
        pygame.time.wait(3000)
        pygame.quit()